In [ ]:
"""

Compare present area with the 2100 and 2150 areas to avoid interpreting things for ice shelves that are already non-viable

"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf

In [ ]:
%matplotlib qt5

In [ ]:
inputpath = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/'

In [ ]:
geo_2025 = xr.open_dataset(inputpath + 'ElmerIce_2025/ElmerIce_4km_2025isf_masks_and_info_and_distance_oneFRIS.nc')
geo_2100 = xr.open_dataset(inputpath + 'ElmerIce_2100/ElmerIce_4km_2100isf_masks_and_info_and_distance_oneFRIS.nc')
geo_2150 = xr.open_dataset(inputpath + 'ElmerIce_2150/ElmerIce_4km_2150isf_masks_and_info_and_distance_oneFRIS.nc')

In [ ]:
inputpath_ElmerIce='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ELMERICE_NEWGEO/'
map_lim = [-3000000,3000000]
file_mask_2100 = uf.cut_domain_stereo(xr.open_dataset(inputpath_ElmerIce+'ElmerIce_4km_allvars_2100.nc'), map_lim, map_lim)
file_mask_2150 = uf.cut_domain_stereo(xr.open_dataset(inputpath_ElmerIce+'ElmerIce_4km_allvars_2150.nc'), map_lim, map_lim)

In [ ]:
file_msk = file_mask_2100['mask_0_1_2']
file_draft_2100 = (file_mask_2100['thickness'] - file_mask_2100['surface']).where(file_msk==1)
file_draft_2100 = file_draft_2100.where(file_draft_2100 > 0, 0)
file_isf_conc_2100 = file_mask_2100['isf_conc']

file_msk = file_mask_2150['mask_0_1_2']
file_draft_2150 = (file_mask_2150['thickness'] - file_mask_2150['surface']).where(file_msk==1)
file_draft_2150 = file_draft_2150.where(file_draft_2150 > 0, 0)
file_isf_conc_2150 = file_mask_2150['isf_conc']


In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]
file_isf = file_isf_nonnan.sel(Nisf=sorted_isf_rignot)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
    

In [ ]:
(file_isf['ISF_mask'] == kisf) 

In [ ]:
(file_draft_2100 <= 10).plot()

In [ ]:
new_area_2100 = []
new_area_2150 = []
for kisf in file_isf.Nisf:
    area_kisf_2100 = (((geo_2100['ISF_mask'] == kisf) & (file_draft_2100 > 10)).astype(int) * file_isf_conc_2100).sum() * (4000 * 4000) * 10**-6
    new_area_2100.append(area_kisf_2100.assign_coords({'Nisf': kisf}))
    area_kisf_2150 = (((geo_2150['ISF_mask'] == kisf) & (file_draft_2150 > 10)).astype(int) * file_isf_conc_2150).sum() * (4000 * 4000)* 10**-6
    new_area_2150.append(area_kisf_2150.assign_coords({'Nisf': kisf}))
new_area_2100_da = xr.concat(new_area_2100, dim='Nisf')
new_area_2150_da = xr.concat(new_area_2150, dim='Nisf')

In [ ]:
new_area_2100_da

In [ ]:
geo_2025['isf_area_here'].load()

In [ ]:

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)

    ax[i].axhline(y=geo_2025['isf_area_here'].sel(Nisf=kisf),color='orange',linewidth=2)
    ax[i].axhline(y=new_area_2100_da.sel(Nisf=kisf),color='red',linewidth=2)
    ax[i].axhline(y=new_area_2150_da.sel(Nisf=kisf),color='magenta',linewidth=2)
    #ax[i].axhline(y=geo_2100['isf_area_here'].sel(Nisf=kisf),color='red',linewidth=2)
    #ax[i].axhline(y=geo_2150['isf_area_here'].sel(Nisf=kisf),color='magenta',linewidth=2)
    
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))


    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

In [ ]:
mask_2025 = geo_2025['ISF_mask'].where(geo_2025['ISF_mask'].isin(sorted_isf_rignot))
mask_2100 = geo_2100['ISF_mask'].where((file_draft_2100 > 10) & (geo_2025['ISF_mask'].isin(sorted_isf_rignot)))
mask_2150 = geo_2150['ISF_mask'].where((file_draft_2150 > 10) & (geo_2025['ISF_mask'].isin(sorted_isf_rignot)))


In [ ]:
plt.figure()
((mask_2150 > 0).astype(int) + (mask_2025 > 0).astype(int)).plot()

In [ ]:
plt.figure()
(mask_2150 > 0).plot()

In [ ]:
((geo_2150['ISF_mask'] == kisf) & (file_draft_2150 > 10)).plot()